# 課題1

別途支給するテスト用画像(136枚)から、ナンバープレートを検出するプログラムを開発してください。

#### 目的



#### 手順

##### 1. テストデータの配置

画像(.jpg)とアノテーション(.txt)のデータセットを訓練用(train)とｔ評価用(val)に分割して本ディレクトリに配置する。


```
datasets
├　images
│　├　train
│　│　└　画像(.jpg)
│　└　val
│　 　└　YOLO形式(.txt)
└　labels
 　├　train
 　│　└　画像(.jpg)
 　└　val
 　 　└　YOLO形式(.txt)
```

##### 2. 訓練実行

以下実行

In [3]:
!python yolov5/train.py --img 640 --batch 16 --epochs 100 --data dataset.yml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=dataset.yml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-378-g2f74455a Python-3.11.10 torch-2.4.1+cu124 CUDA:0 (NVIDIA RTX A4000, 16102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0

# 課題2

検出したナンバープレートにBounding Boxを描画した画像を出力してください。

# 課題3

黄色のナンバープレート(軽自動車)の件数を出力してください。